In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
def custom_activation(x):
    return x * torch.sigmoid(x)

In [3]:
class ModifiedLeNet(nn.Module):
    def __init__(self):
        super(ModifiedLeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=3, padding=1)  
        self.conv2 = nn.Conv2d(6, 16, kernel_size=3)  
        self.fc1 = nn.Linear(16*6*6, 120)  
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = custom_activation(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = custom_activation(self.conv2(x))
        x = F.max_pool2d(x, 2)
        
        x = torch.flatten(x, 1) 
        x = custom_activation(self.fc1(x))
        x = custom_activation(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)

In [4]:
train_folder = r'C:\Users\Aaryan\Downloads\MNIST Dataset JPG format\MNIST Dataset JPG format\MNIST - JPG - training'
test_folder = r'C:\Users\Aaryan\Downloads\MNIST Dataset JPG format\MNIST Dataset JPG format\MNIST - JPG - testing'

In [5]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [6]:
train_dataset = datasets.ImageFolder(root=train_folder, transform=transform)
test_dataset = datasets.ImageFolder(root=test_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModifiedLeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1, Loss: 1.5799
Epoch 2, Loss: 1.4913
Epoch 3, Loss: 1.4829
Epoch 4, Loss: 1.4792
Epoch 5, Loss: 1.4766
Epoch 6, Loss: 1.4745
Epoch 7, Loss: 1.4732
Epoch 8, Loss: 1.4731
Epoch 9, Loss: 1.4715
Epoch 10, Loss: 1.4713


In [8]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 98.72%
